In [1]:
import itertools

import editdistance
import pandas as pd
from nltk import ngrams
from tqdm.auto import tqdm

In [2]:
WORDS = set(pd.read_csv('../List_3/data/words.txt', sep=";", header=None)[0])

In [18]:
def preprocess(word):

    new_word = ""
    final_word = ""
    s1 = {"ó":"o",
     "ę": "e",
     "ą": "a",
     "ć": "c",
     "ź": "z",
     "ż": "z"}
    s2 = {"rz": "ż",
     "ch": "h"}
    for i in range(len(word)):
        if word[i] in s1:
            new_word += s1[word[i]]
        else:
            new_word += word[i]
    for i in range(len(new_word)):
        if (i+1 < len(new_word)) and (new_word[i] + new_word[i+1] in s2):
            final_word += s2[new_word[i] + new_word[i+1]]
        else:
            final_word += new_word[i]
    final_word = ''.join(c[0] for c in itertools.groupby(final_word))
    return final_word

In [19]:
ERRORS = pd.read_csv('../List_3/data/literowki1.txt', sep=" ", header=None, names=['correct', 'error'])

In [20]:
N_GRAMS = {word: set(map("".join, ngrams(word, 2))) for word in WORDS}

In [45]:
from  nltk.metrics.distance import edit_distance

def correct_error_2(word):
    if word in WORDS:
        return [word]
    else:
        processed_word = preprocess(word)
        possible_words = []
        word_ngrams = set(map("".join, ngrams(word, 2)))
        for correct_word, correct_ngrams in N_GRAMS.items():
            if len(word_ngrams & correct_ngrams) > 0.40 * len(word_ngrams):
                possible_words.append(correct_word)
        min_editdist = 100
        best_word = []
        for correct_word in possible_words:
            editdist = (editdistance.eval(preprocess(correct_word), processed_word))/len(word)
            if editdist < min_editdist:
                min_editdist = editdist
                best_word = [correct_word]
            elif editdist < min_editdist + 2:
                best_word.append(correct_word)
        return best_word

In [48]:
def find_from_best_1(w, best):
    scores = {}
    w = preprocess(w)
    for b in best:
        cb = b
        b = preprocess(b)
        i = 0
        scores[cb] = 0
        while b[i] == w[i]:
            scores[cb] += 1
            i+= 1
            if i == min(len(b), len(w)):
                break
        i = -1
        tries = 1
        while b[i] == w[i] or tries:
            if b[i] != w[i]:
                tries = 0
            scores[cb] += 1
            i -= 1
            if i == -min(len(b), len(w)):
                break
        scores[cb] -= abs(len(b) - len(w))
    s = sorted(scores, key=scores.get, reverse=True)
    return s[0], s

In [49]:
acc = 0
r = 1
for i, (correct, error) in tqdm(ERRORS[::-1].iterrows()):
    best = correct_error_2(error)
    corrected, scores = find_from_best_1(error, best)
    if corrected == correct:
        acc += 1
    else:
        print(corrected, correct, error)
    print(acc / r)
    r+=1
print(100 * acc/len(ERRORS))

0it [00:00, ?it/s]

1.0
przypuszczać podpuszczać pudpudszczać
0.5
pospuszczać podpuszczać posdupszcać
0.3333333333333333
poduszczać podpuszczać podpszżczać
0.25
0.4
0.5
werdragonowy wyjątkowego wyjaktnowego
0.42857142857142855
wyjątkowo wyjątkowego wyjatkoweggo
0.375
bezwyjątkowy wyjątkowego wujątkowmego
0.3333333333333333
wyjątkowo wyjątkowego wyjatkowmego
0.3
wydatkowość wyjątkowego wyatkowrego
0.2727272727272727
0.3333333333333333
0.38461538461538464
0.42857142857142855
0.4666666666666667
niezamazaność ślamazarność slamanzanosc
0.4375
0.47058823529411764
0.5
0.5263157894736842
0.55
niezamazaność ślamazarność slamarazanosc
0.5238095238095238
kontrafakcja komunikacja konuniakcaja
0.5
0.5217391304347826
0.5416666666666666
komunizacja komunikacja komunijajca
0.52
kominiarka komunikacja kominiakaca
0.5
0.5185185185185185
0.5357142857142857
organek ogródek orgudek
0.5172413793103449
0.5333333333333333
0.5483870967741935
0.5625
kalatea kalarepa kalapea
0.5454545454545454
0.5588235294117647
0.5714285714285714
